In [5]:
import numpy as np
import time
import datetime
import random
import pandas as pd

In [6]:
class Calisan:
    def __init__(self, TC_NO, isim, soyisim, adres, tel_no, e_posta) -> None:
        self.TC_NO = TC_NO
        self.isim = isim
        self.soyisim = soyisim
        self.adres = adres
        self.tel_no = tel_no
        self.e_posta = e_posta

In [7]:
def unique_time():
    time.sleep(0.1)
    return round(time.time() * 10000000) % 1000000

In [8]:
def TCNOGenerator():
    st = time.gmtime()
    TCNO = str(unique_time())
    while (len(TCNO) < 11):
        TCNO = TCNO + str(random.randint(0, 9))
    return TCNO

In [9]:
def TCNOGeneratorN(n):
    arr = np.zeros(n, dtype='U11')
    for i in range(n):
        arr[i] = TCNOGenerator()
    return arr

In [10]:
def createPerson(n):
    ogretmen_tcnos = df_tcnos.tail(n)['id'].to_numpy()
    df_tcnos.drop(df_tcnos.tail(n).index, inplace=True)
    ogretmen_names = df_names.sample(n = n)['Name'].to_numpy()
    ogretmen_surnames = df_surnames.sample(n = n)['Surname'].to_numpy()
    ogretmen_adres = ogretmen_names + ogretmen_surnames
    ogretmen_telno = ogretmen_tcnos
    ogretmen_eposta = np.empty(len(ogretmen_tcnos), dtype='U20')
    for i in range(len(ogretmen_tcnos)):
        ogretmen_eposta[i] = str(ogretmen_tcnos[i]) + '@mail.com'
    return pd.DataFrame({'TC_No': ogretmen_tcnos, 'Isim': ogretmen_names, 'Soyisim': ogretmen_surnames, 'Adres': ogretmen_adres, 'Tel_no': ogretmen_telno, 'E_posta': ogretmen_eposta})

In [11]:
def createTemizlikci(n):
    return createPerson(n)

In [12]:
def createIdari(n):
    return createPerson(n)

In [13]:
def CreateOgrenciVeli(n):
    df_ogrenci = createPerson(n)
    df_veli = createPerson(n)
    df_ogrenci.drop('Adres', axis=1, inplace=True)
    df_veli.drop('Adres', axis=1, inplace=True)
    oTC = df_ogrenci['TC_No'].to_numpy()
    vTC = df_veli['TC_No'].to_numpy()
    relation = np.empty(len(oTC), dtype='U4')
    for i in range(len(oTC)):
        relation[i] = 'anne'
    df_relation = pd.DataFrame({'Vtc_No': vTC, 'Otc_No': oTC, 'yakinlik': relation})
    return df_ogrenci, df_veli, df_relation

In [14]:
def CreateMezunOgrenciVeli(n):
    df_ogrenci, df_veli, df_relation = CreateOgrenciVeli(n)
    date = np.empty(df_ogrenci.shape[0])
    for i in range(df_ogrenci.shape[0]):
        date[i] = 0
    df_ogrenci['Mezuniyet_tarihi'] = date
    return df_ogrenci, df_veli, df_relation

In [15]:
def createFullTimeOgretmen(n):
    df_ogretmen = createPerson(n)
    part_mi = np.empty(n, dtype=np.bool_)
    maas = np.empty(n, dtype=np.uint16)
    for i in range(n):
        part_mi[i] = False
        maas[i] = random.randint(1000, 2000)
    df_ogretmen['Part_mi'] = part_mi
    df_ogretmen['Maas'] = maas
    return df_ogretmen

In [16]:
def createPartTimer(n):
    df_ogretmen = createPerson(n)
    part_mi = np.empty(n, dtype=np.bool_)
    saat = np.empty(n, dtype=np.uint8)
    for i in range(n):
        part_mi[i] = True
        saat[i] = random.randint(1, 20)
    df_ogretmen['Part_mi'] = part_mi
    df_ogretmen['Saat'] = saat
    return df_ogretmen

In [17]:
df_names = pd.read_csv('names.txt')
df_surnames = pd.read_csv('surnames.txt')
df_tcnos= pd.read_csv('tcnos2.csv')

In [18]:
# Çalışan datası oluşturma

calisan = pd.DataFrame()

In [19]:
df_temizlikci = createTemizlikci(5)
df_idari = createIdari(2)
df_part_timer = createPartTimer(5)
df_full_time = createFullTimeOgretmen(10)

df_calisan = pd.concat([df_temizlikci, df_idari, df_part_timer.drop('Saat', axis=1), df_full_time.drop('Maas', axis=1)])
df_calisan.drop(['Part_mi'], axis=1, inplace=True)
df_ogretmen = pd.concat([df_part_timer.drop('Saat', axis=1), df_full_time.drop('Maas', axis=1)])
df_full_time.drop(['Part_mi'], axis=1, inplace=True)
df_part_timer.drop(['Part_mi'], axis=1, inplace=True)

df_aogrenci, df_aveli, df_arelation = CreateOgrenciVeli(500)
df_mogrenci, df_mveli, df_mrelation = CreateMezunOgrenciVeli(100)
df_ogrenci = pd.concat([df_aogrenci, df_mogrenci.drop('Mezuniyet_tarihi', axis=1)])
df_veli = pd.concat([df_aveli, df_mveli])
df_relation = pd.concat([df_arelation, df_mrelation])

In [20]:
df_ucret = pd.DataFrame()
saatArray = np.unique(df_part_timer['Saat'].to_numpy())
ucretArray = np.empty(len(saatArray), dtype=np.uint16)
for i in range(len(saatArray)):
    ucretArray[i] = saatArray[i] * 10

df_ucret['Saat'] = saatArray
df_ucret['Ucret'] = ucretArray

In [24]:
df_temizlikci.loc[:, 'TC_No'].to_csv('data/Temizlikci.csv', index=False)
df_idari.loc[:, ['TC_No']].to_csv('data/Idari.csv', index=False)
df_part_timer.loc[:, ['TC_No', 'Saat']].to_csv('data/PartTimer.csv', index=False)
df_full_time.loc[:, ['TC_No', 'Maas']].to_csv('data/FullTimer.csv', index=False)
df_calisan.to_csv('data/Calisan.csv', index=False)
df_ogretmen.loc[:, ['TC_No', 'Part_mi']].to_csv('data/Ogretmen.csv', index=False)

df_ucret.to_csv('data/Ucret_Tablosu.csv', index=False)

df_ogrenci.to_csv('data/Ogrenci.csv', index=False)
df_aogrenci.loc[:, ['TC_No']].to_csv('data/Aktif.csv', index=False)
df_mogrenci.loc[:, ['TC_No', 'Mezuniyet_tarihi']].to_csv('data/Mezun.csv', index=False)
df_veli.to_csv('data/Veli.csv', index=False)
df_relation.to_csv('data/Aile_Iliskisi.csv', index=False)

In [25]:
import pandas as pd

# Function to read CSV file and generate SQL insert queries
def csv_to_sql(csv_file, table_name):
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    columns = ', '.join(str(column).upper() for column in df.columns)

    # Generate SQL insert queries
    sql_query = ""
    for row in df.to_numpy():
        values = ', '.join([f"'{str(value)}'" for value in row])
        sql_query = sql_query + f"INSERT INTO {table_name} ({columns}) VALUES ({values});\n"

    return sql_query

# List of CSV file names and corresponding table names
csv_table_mapping = {
    'Calisan.csv': 'calisan',
    'Temizlikci.csv': 'temizlikci',
    'Ogretmen.csv': 'ogretmen',
    'Aile_Iliskisi.csv': 'aile_iliskisi',
    'Aktif.csv': 'aktif',
    'FullTimer.csv': 'full_timer',
    'Idari.csv': 'idari',
    'Mezun.csv': 'mezun',
    'Ogrenci.csv': 'ogrenci',
    'Veli.csv': 'veli',
    'PartTimer.csv': 'part_timer',
    'Ucret_Tablosu.csv': 'ucret_tablosu'
}

# Directory path where CSV files are located
csv_directory = './data'

# Iterate through the CSV files and generate SQL queries
for csv_file, table_name in csv_table_mapping.items():
    file_path = f'{csv_directory}/{csv_file}'
    sql_query = csv_to_sql(file_path, table_name)

    text_file = open("./insertQuery/" + table_name + ".txt", "w")

    n = text_file.write(sql_query)

    if n == len(sql_query):
        print("Success! String written to text file.")
    else:
        print("Failure! String not written to text file.")

    # Close file
    text_file.close()


Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.
Success! String written to text file.


In [3]:
import pandas as pd
df = pd.read_csv("./Data/Aktif.csv")